# TP1 - Grupo 14

## Problema 1 - Horário

Pretende-se construir um horário semanal para o plano de reuniões de projeto de uma “StartUp” de acordo com as seguintes condições:

    1. Cada reunião ocupa uma sala (enumeradas 1...S,) durante um “slot”  1..T\, (hora,dia).  
    2. Cada reunião tem associado um projeto (enumerados 1..P) e um conjunto de participantes. Os diferentes colaboradores são enumerados 1..C.
    3. Cada projeto tem associado um conjunto de colaboradores, dos quais um  é o líder. Cada projeto realiza um dado número de reuniões semanais. 
    4. O líder do projeto participa em todas as reuniões do seu projeto; os restantes colaboradores podem ou não participar consoante a sua disponibilidade, num mínimo (“quorum”) de  50% do total de colaboradores do projeto. 

# Implementação
Para a resolução deste problema iremos utilizar a biblioteca do OR-Tools, mais precisamente a ferramente pywraplp

In [1]:
from ortools.linear_solver import pywraplp

## Inputs
Nesta secção do trabalho vamos dar exemplos de alguns inputs possíveis e não possíveis para que consigamos melhor entender a nossa linha de código, cada input vai ser constituído pelos parâmetros S salas, T intervalos de tempo, P projetos e C colaboradores, tanto como um dicionário que descreve cada projeto e por fim uma matriz de disponibilidade de cada colaborador.

#### Exemplo 1 

In [2]:
T, S, P, C = 3, 2, 3, 5    #período de 8 horas; 2 salas

projetos = {}
projetos[0] = {'colaboradores':[0,1,2,4], 'lider':4, 'min':1}
projetos[1] = {'colaboradores':[1,2,3], 'lider':1, 'min':2}
projetos[2] = {'colaboradores':[0,2,3,4], 'lider':3, 'min':2}

matrix = {}
matrix[0] = [1,0,1]
matrix[1] = [0,1,1]
matrix[2] = [1,1,0]
matrix[3] = [1,1,1]
matrix[4] = [0,1,1]


Para começar o programa, vamos definir a função scheduleStartUp que recebe os parâmetros de S salas, T intervalos de tempo, P projetos e C colaboradores, o dicionário de projetos e a matriz de disponibilidade de cada colaborador.
Dentro da função tem definida a matriz de alocação xDict como um dicionário e já incluída uma restrição de que "Para cada intervalos de tempo, para cada sala e para cada projeto, terá que haver unm líder em  cada projeto", podendo matematicamente ser traduzida como, a soma dos líderes (l) em cada projeto, sala e intervalo de tempo é 1:

$$ \forall_{t<T} \cdot \forall_{s<S} \cdot \forall_{p<P} \cdot \sum x_{l,t,s,p} == 1$$

In [3]:
def scheduleStartUp(S, T, P, C, projDict, dispMatrix):
    xDict = {}
    result = dict()
    solver = pywraplp.Solver.CreateSolver('SCIP')
    #dicionário com T*P*C*S variáveis -> xDict[t][p][c][s] == 1 sse c estará presente na reunião do projeto p no timeslot t na sala s.
    for t in range(T):
        xDict[t] = {}
        for s in range(S):
            xDict[t][s] = {}
            for p in range(P):
                if dispMatrix[projDict[p]['lider']][t] == 1:
                    xDict[t][s][p] = {}
                    for c in projDict[p]['colaboradores']:
                        if dispMatrix[c][t] == 1:
                            xDict[t][s][p][c] = solver.BoolVar('x[%i][%i][%i}[%i]' % (t,s,p,c))

### Restrições

##### 1. A restrição "Um colaborador pode ter no máximo uma reunião por cada intervalo de tempo" pode ser expressa por:
$$\forall_{t<T} \cdot \forall_{c<C} \cdot \sum x_{s,p} \le 1$$

In [4]:
for t in xDict:
    for c in range(C):
        solver.Add(sum([xDict[t][s][p][c] for s in xDict[t] for p in xDict[t][s] if c in xDict[t][s][p]]) <= 1)

NameError: name 'xDict' is not defined

##### 2. A restrição "Cada líder tem de participar no seu projeto" alberga também a restrição "Cada colaborador participa apenas de um projeto se o seu líder participar também" e pode ser traduzida matematicamente como:
$$ \forall_{t<T} \cdot \forall_{s<S} \cdot \forall_{p<P} \cdot \forall_{c<C} \cdot x_{t,x,p,c} \le x_{t,s,p,l}$$

Ou seja, o colaborador está na reunião de um líder ou simplesmente não está (1 ou 0).

In [5]:
for t in xDict:
    for s in xDict[t]:
        for p in xDict[t][s]:
            for c in xDict[t][s][p]:
                solver.Add(xDict[t][s][p][c] <= xDict[t][s][p][projDict[p]['lider']])

NameError: name 'xDict' is not defined

3. "Uma reunião apenas acontece se pelo menos 50% de todos os colaboradores desse projeto se encontram presentes nela e se o líder fizer parte dessa metade"

Esta restrição pode ser expressa como:
$$ \forall_{t<T} \cdot \forall_{p<P} \cdot \forall_{s<S} \cdot \sum_{c<C} x_{t,s,p,c} \ge \sum_{c<C} \frac{{x_{t,s,p,c}}}{\text{2}} * x_{t,s,p,l}$$

Que, para o nosso programa, usaremos as regras de cálculo (com o intuito de evitar tipos floats e ints na mesma linha de código) para tornar a expressão em:

$$ \forall_{t<T} \cdot \forall_{p<P} \cdot \forall_{s<S} \cdot 2 * \sum_{c<C} x_{t,s,p,c} \ge \sum_{c<C} x_{t,s,p,c} * x_{t,s,p,l}$$

In [ ]:
for t in xDict:
    for p in range(P):
            for s in xDict[t]:
                if p in xDict[t][s]:
                    solver.Add(2*sum([xDict[t][s][p][c] for c in xDict[t][s][p]]) >= 
                               len(projDict[p]['colaboradores']) * xDict[t][s][p][projDict[p]['lider']])

4. Cada projeto tem um número míninimo de reuniões semanais já incluídas no seu input, ou seja, o número de reuniões em que o líder se encontra tem que ser pelo menos o mesmo que o valor 'min':
$$ \forall_{t<T} \cdot \forall_{p<P} \cdot \sum_{s<S,c<C} x_{t,s,p,l} \ge min $$

In [6]:
for t in xDict:
    for p in range(P):
        solver.Add(sum([xDict[t][s][p][projDict[p]['lider']] for t in xDict for s in xDict[t] if p in xDict[t][s]]) >= projDict[p]['min'])

NameError: name 'xDict' is not defined

## Critérios de otimização

1.Maximizar o número de reuniões efetivamente realizadas

In [8]:
solver.Maximize(sum([xDict[t][s][p][projDict[p]['lider']] for t in xDict for s in xDict[t] for p in xDict[t][s]]))

NameError: name 'solver' is not defined

2.Minimizar o número médio de reuniões por participante.

In [ ]:
solver.Minimize(sum([xDict[t][s][p][c] for t in xDict for s in xDict[t] for p in xDict[t][s] for c in xDict[t][s][p]]))

## Resolução dos problemas

Com todas as restrições impostas ao solver, apenas é necessário procurarmos uma solução, se não existir uma, o programa dirá que não é possível.

In [ ]:

    status = solver.Solve()
    
    if status == pywraplp.Solver.OPTIMAL:
        for t in xDict:
            print(f'Slot: {t}')
            for s in xDict[t]:
                for p in xDict[t][s]:
                    if projDict[p]['lider'] in xDict[t][s][p] and xDict[t][s][p][projDict[p]['lider']].solution_value() == 1:
                        print(f'Projeto: {p} Sala: {s}')
                        print("Colaboradores:")
                        for c in xDict[t][s][p]:
                            if xDict[t][s][p][c].solution_value() == 1:
                                print(f'{c}')
                        print("")
            print("")
        print(solver.Objective())
        print(solver.Objective().Value())
    else:
        print("Nao é possivel construir o hórario")
        
scheduleStartUp(S, T, P, C, projetos, matrix)